In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchaudio.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import torch.nn.functional as F
import string
import re
import sys

from datetime import datetime

In [ ]:
!pip install datasets -q

# Get the data

## Download dataset

In [ ]:
from datasets import load_dataset

ds = load_dataset("Seferovic/bosnian-news-articles-dataset-from-klixba")

In [ ]:
df = ds['train'].to_pandas()

In [ ]:
df.head()

,title,link,article_class,article_class_name,num_of_comments,num_of_shares,picture_path,text
0,Ukrajinski piloti započeli obuku za upravljanj...,https://www.klix.ba/vijesti/svijet/ukrajinski-...,vijesti,Obučavaju ih Amerikanci,0,0,https://static.klix.ba/media/images/vijesti/b_...,Ukrajinski piloti započeli su zajedničku obuku...
1,Košarkaši BiH se danas protiv Poljske bore za ...,https://www.klix.ba/sport/kosarka/kosarkasi-bi...,sport,Finale pretkvalifikacija,8,0,https://static.klix.ba/media/images/vijesti/b_...,Košarkaška reprezentacija Bosne i Hercegovine ...
2,Nakon više od 80 godina Kaliforniji se sprema ...,https://www.klix.ba/vijesti/svijet/nakon-vise-...,vijesti,Uragan Hilary,17,18,https://static.klix.ba/media/images/vijesti/b_...,Uragan Hilary koji se kreće prema pacifičkoj o...
3,Kremlj je na popis stranih agenata u Rusiji uv...,https://www.klix.ba/vijesti/svijet/kremlj-je-n...,vijesti,Paranoja u Moskvi,4,27,https://static.klix.ba/media/images/vijesti/23...,Rusko ministarstvo pravde uključilo je na tako...
4,Savo Manojlović odgovorio Ani Brnabić: Da li s...,https://www.klix.ba/vijesti/regija/savo-manojl...,vijesti,Pitanje odgovornosti,8,8,https://static.klix.ba/media/images/vijesti/b_...,"Direktor pokreta ""Kreni-Promeni"" Savo Manojlov..."


In [ ]:
text = df['text'].values[:1500]

In [ ]:
text = ''.join(text)
text = text[:1_050_000].lower()

In [ ]:
len(text)

1050003

## Prepare dataset

In [ ]:
accepted_text= string.ascii_lowercase + 'šđžčć' + string.digits + '!?.,"()+-/@%–' + "':" + ' ' + '\n'
chars = [x for x in accepted_text]
print(chars)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'š', 'đ', 'ž', 'č', 'ć', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '!', '?', '.', ',', '"', '(', ')', '+', '-', '/', '@', '%', '–', "'", ':', ' ', '\n']


In [ ]:
text = ''.join([char for char in text if char in accepted_text])

In [ ]:
test_size = 0.20
train_text = text[:int(len(text)*(1-test_size))]
test_text = text[int(len(text)*(1-test_size)):]

In [ ]:
len(train_text), len(test_text)

(838059, 209515)

## Create vocab

In [ ]:
vocab={}

In [ ]:
for i, char in enumerate(chars):
  vocab[char] = i

In [ ]:
vocab_size = len(vocab)
print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 58


## Create Custom tokenizer

In [ ]:
class Tokenizer(object):
  def __init__(self,vocabulary):
    self.vocabulary=vocabulary

  def __call__(self,sequence, label=False):
    sequence = sequence.lower()
    if label:
      token = F.one_hot(torch.tensor(self.vocabulary[sequence]),num_classes=len(self.vocabulary))
      return token.type(torch.float32)
    tokens =[]
    for letter in sequence:
      tokens.append(self.vocabulary[letter])
    return torch.tensor(tokens)

In [ ]:
tokenizer = Tokenizer(vocab)

## Create Custom dataset object

In [ ]:
class TextDataset(Dataset):
  def __init__(self,text,T,tokenizer=None):
    self.tokenizer = tokenizer
    self.text = text
    self.T = T


  def __len__(self):
    return len(self.text)-self.T

  def __getitem__(self,idx):
    x = self.text[idx:idx+self.T]
    y = self.text[idx+self.T]

    if self.tokenizer:
      x = self.tokenizer(x)
      y = self.tokenizer(y, label=True)


    #x = x.astype(np.float32)
   # y = np.array(y).reshape(-1,1).astype(np.float32)
    return x.long(), y

## Load datasets

In [ ]:
train_dataset = TextDataset(train_text,150,tokenizer=tokenizer)
test_dataset = TextDataset(test_text,150,tokenizer=tokenizer)

In [ ]:
len(train_dataset), len(test_dataset)

(837909, 209365)

In [ ]:
for i,k in train_dataset:
  print(i,k)
  break

tensor([20, 10, 17,  0,  9,  8, 13, 18, 10,  8, 56, 15,  8, 11, 14, 19,  8, 56,
        25,  0, 15, 14, 29,  4, 11,  8, 56, 18, 20, 56, 25,  0,  9,  4,  3, 13,
         8, 29, 10, 20, 56, 14,  1, 20, 10, 20, 56, 25,  0, 56, 20, 15, 17,  0,
        21, 11,  9,  0, 13,  9,  4, 56,  1, 14, 17,  1,  4, 13,  8, 12, 56,  0,
        21,  8, 14, 13,  8, 12,  0, 56,  5, 49, 32, 37, 56, 18,  0, 56,  0, 12,
         4, 17,  8, 29, 10,  8, 12, 56,  8, 13, 18, 19, 17, 20, 10, 19, 14, 17,
         8, 12,  0, 56, 20, 56, 20, 10, 17,  0,  9,  8, 13,  8, 43, 56, 14, 21,
        20, 56,  8, 13,  5, 14, 17, 12,  0,  2,  8,  9, 20, 56,  9,  4, 56, 25,
         0, 56, 12,  4,  3,  8]) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])


## Initilize DataLoaders

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=32, drop_last=True)

In [ ]:
for inputs, targets in train_loader:
  print('inputs: ', inputs, 'shape: ', inputs.shape)
  print('targets: ', targets, 'shape: ', targets.shape)
  break

inputs:  tensor([[ 9,  4, 56,  ...,  4, 56,  8],
        [ 0, 21, 13,  ..., 30,  4, 56],
        [ 9,  0,  9,  ..., 21,  4, 30],
        ...,
        [18, 19,  0,  ...,  0, 21, 13],
        [ 8, 56, 18,  ..., 18, 12,  0],
        [56, 25,  0,  ..., 11,  8, 10]]) shape:  torch.Size([32, 150])
targets:  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) shape:  torch.Size([32, 58])


# Create the model

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
class RNN(nn.Module):
  def __init__(self,vocab_size,embed_size,n_hidden,n_layers,n_outputs):
    super(RNN,self).__init__()
    self.V = vocab_size
    self.D = embed_size
    self.M = n_hidden
    self.L = n_layers
    self.K = n_outputs

    self.embedding = nn.Embedding(self.V,self.D)

    # Initalize rnn and fc layers
    self.rnn = nn.LSTM(input_size=self.D,
                      hidden_size=self.M,
                      num_layers=self.L,
                      dropout=0.5,
                      batch_first=True)

    self.fc = nn.Sequential(
          nn.Linear(self.M, self.K),
         # nn.ReLU(),
         # nn.Linear(1024, 512),
         # nn.ReLU(),
         # nn.Linear(512,self.K)
        )

  def forward(self, X, hidden):
    # Embedding layer:

    out = self.embedding(X)
    # pass through rnn

    out,hidden_state= self.rnn(out,hidden)
   # out = F.relu(out)
    out = self.fc(out[:,-1,:])
    return out,(hidden_state[0].detach(), hidden_state[1].detach())


In [ ]:
model = RNN(vocab_size=vocab_size,
            embed_size=4,
            n_hidden=512,
            n_layers=3,
            n_outputs=vocab_size)
model.to(device)

RNN(
  (embedding): Embedding(58, 4)
  (rnn): LSTM(4, 512, num_layers=3, batch_first=True, dropout=0.5)
  (fc): Sequential(
    (0): Linear(in_features=512, out_features=58, bias=True)
  )
)

In [ ]:
len(vocab)

58

In [ ]:
for i, k in train_loader:
  print(i.shape)
  tada,_ = model(i.to(device), None)
  print(tada.size())
  break

torch.Size([32, 150])
torch.Size([32, 58])


# Train model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
from helper_functions import progress_bar, plot_loss_curves,SaveModelCheckpoint

In [ ]:
save_model_checkpoint = SaveModelCheckpoint(path="model3_checkpoint.pt")
best_val_loss=float('inf')

In [ ]:
epoches=30
train_losses = np.zeros(epoches)
val_losses = np.zeros(epoches)
for it in range(epoches):
  t0 = datetime.now()
  current_batch = 0
  total_batches = len(train_loader)
  model.train() # set model to train mode
  train_loss=[]
  val_loss=[]
  hidden_state = None
  # train
  for inputs,targets in train_loader:
    # move data to gpu
    inputs,targets = inputs.to(device),targets.to(device)
    #inputs = inputs.permute(0,2,1)
    # zero gradients
    optimizer.zero_grad()

    # forward pass
    outputs,hidden_state = model(inputs,hidden_state)
    loss = criterion(outputs,targets)

    # backward
    loss.backward()
    optimizer.step()

    train_loss.append(loss.item())
    current_batch = progress_bar(current_batch,total_batches)

  model.eval() # set model to eval mode
  current_batch = 0
  total_batches = len(test_loader)
  for inputs,targets in test_loader:
    # move data to gpu
    inputs,targets = inputs.to(device),targets.to(device)
   # inputs = inputs.permute(0,2,1)


    # forward pass
    outputs,hidden_state = model(inputs,hidden_state)
    loss = criterion(outputs,targets)

    val_loss.append(loss.item())
    current_batch = progress_bar(current_batch,total_batches,validation=True)


  # calculate loss
  print('\r')
  train_loss = np.mean(train_loss)
  val_loss = np.mean(val_loss)
  best_val_loss=  save_model_checkpoint(val_loss,best_val_loss,train_loss,it, model=model, optimizer=optimizer)

  # append loss
  train_losses[it]=train_loss
  val_losses[it]=val_loss
  dt = datetime.now() - t0
  print(f"Epoch {it+1}/{epoches}, Train loss: {train_loss:.4f}, Val loss: {val_loss:.4f}, Duration: {dt}")
  print('-------------------------------------------------------------')

Validation: Batch 6542/6542 - [==================================================]
Model saved at epoch: 1, val_loss improved from: inf to: 2.2146
Epoch 1/30, Train loss: 2.5928, Val loss: 2.2146, Duration: 0:13:06.850978
-------------------------------------------------------------
Validation: Batch 6542/6542 - [==================================================]
Model saved at epoch: 2, val_loss improved from: 2.2146 to: 1.7368
Epoch 2/30, Train loss: 1.9021, Val loss: 1.7368, Duration: 0:13:04.222969
-------------------------------------------------------------
Validation: Batch 6542/6542 - [==================================================]
Model saved at epoch: 3, val_loss improved from: 1.7368 to: 1.6533
Epoch 3/30, Train loss: 1.7168, Val loss: 1.6533, Duration: 0:13:02.575257
-------------------------------------------------------------
Validation: Batch 6542/6542 - [==================================================]
Model saved at epoch: 4, val_loss improved from: 1.6533 to:

In [ ]:
print_second = False
if print_second:
  plot_loss_curves(train_losses2,val_losses2,train_losses,val_losses,)
else:
  plot_loss_curves(train_losses,val_losses)
  train_losses2,val_losses2 = train_losses,val_losses

In [ ]:
model.eval()
# train accuracy
n_correct=0
n_total=0
for inputs,targets in train_loader:
  # move data to gpu
  inputs,targets = inputs.to(device),targets.to(device)
  #inputs = inputs.permute(0,2,1)

  # make prediction
  outputs = model(inputs)
  _,predictions = torch.max(outputs,1)
  targets = torch.argmax(targets, dim=1)
  # update counts
  n_correct+=(predictions==targets).sum().item()
  n_total+=targets.shape[0]
train_acc = n_correct/n_total

# test accuracy
n_correct=0
n_total=0
for inputs,targets in test_loader:
  # move data to gpu
  inputs,targets = inputs.to(device),targets.to(device)
  #inputs = inputs.permute(0,2,1)

  # make prediction
  outputs = model(inputs)
  _,predictions = torch.max(outputs,1)
  targets = torch.argmax(targets, dim=1)
  # update counts
  n_correct+=(predictions==targets).sum().item()
  n_total+=targets.shape[0]
test_acc = n_correct/n_total

print(f"Train accuracy: {train_acc:.4f}, Test accuracy: {test_acc:.4f}")

In [ ]:
if True:
    model = RNN(vocab_size=vocab_size,
                embed_size=4,
                n_hidden=512,
                n_layers=3,
                n_outputs=vocab_size)
    model.to(device)
    model.to(device)
    model.load_state_dict(torch.load('model3_checkpoint.pt',map_location=device)['model_state_dict'])
    model.eval()

In [ ]:
vocabulary = {y: x for x, y in vocab.items()}

In [ ]:
import time

In [ ]:
tekst = text[500000:500100]
hidden_state=None
tekst='FK Željezničar je uoči sjednice obavijestio medije da'
#tekst = '''Apsolutni junak Zmajeva večeras je bio golman Nikola Vasilj, koji je sa pet izvanrednih intervencija sačuvao svoju mrežu netaknutom, pa je najzaslužniji za osvojeni bod našeg nacionalnog tima.#
#Od početka utakmice inicijativu je imala selekcija Mađarske, ali ipak se dugo čekalo na prava uzbuđenja jer smo gledali uspavanku na terenu.'''
for i in range(50):
  data = tokenizer(tekst[-100:])
  data = data.reshape(1,-1)
  data = data.to(device)
  outputs,hidden_state = model(inputs,hidden_state)
  out = torch.argmax(outputs,1)
  new_letter = vocabulary[out.cpu().numpy()[0]]
  tekst = tekst+new_letter
  print(tekst)
  time.sleep(0.5)
  print('---------------------------------------------------')
#print(tekst)

FK Željezničar je uoči sjednice obavijestio medije dai
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije daii
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije daiii
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije daiiii
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije daiiiii
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije daiiiiii
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije daiiiiiii
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije daiiiiiiii
---------------------------------------------------
FK Željezničar je uoči sjednice obavijestio medije daiiiiiiiii
---------------------------------------------------
F

In [ ]:
def next_char(text, hidden_state,temperature=1):
    # Predict using the model (assuming text is already processed into the right tensor format)
    with torch.no_grad():
      data = tokenizer(text)
      data = data.reshape(1,-1)
      data = data.to(device)
      logits,hidden_state = model(data,hidden_state)  # Replace with proper text input processing

    # Get the logits for the last predicted character
    #logits = y_proba[:, -1, :]  # Assuming y_proba has shape [batch_size, seq_len, vocab_size]

    # Rescale logits using temperature
    rescaled_logits = logits / temperature

    # Apply softmax to get probabilities and then sample from the categorical distribution
    probabilities = F.softmax(rescaled_logits, dim=-1)
    char_id = torch.multinomial(probabilities, num_samples=1).item()

    # Get the vocabulary and return the corresponding character
    return vocabulary[char_id], hidden_state

In [ ]:
def extend_text(text, n_chars=200, temperature=1):
    hidden_state=None
    class bcolors:
        HEADER = '\033[95m'
        OKBLUE = '\033[94m'
        OKCYAN = '\033[96m'
        OKGREEN = '\033[92m'
        WARNING = '\033[93m'
        FAIL = '\033[91m'
        ENDC = '\033[0m'
        BOLD = '\033[1m'
        UNDERLINE = '\033[4m'

    first_len=len(text)
    for _ in range(n_chars):
        char,hidden_state = next_char(text, hidden_state,temperature)
        text+=char
    print(f"{bcolors.OKGREEN}{text[:first_len]}{bcolors.ENDC}{text[first_len:]}")

    #return text

In [ ]:
telst = "Osim toga, u biltenu MUP-a KS se navodi da su u toku proteklog dana, službenici policije, preduzimajući redovne mjere kontrole saobraćaja, uručili su 554 prekršajna naloga, iz saobraćaja je isključeno 19 vozača zbog up"

In [ ]:
extend_text(telst, temperature=0.5)

Osim toga, u biltenu MUP-a KS se navodi da su u toku proteklog dana, službenici policije, preduzimajući redovne mjere kontrole saobraćaja, uručili su 554 prekršajna naloga, iz saobraćaja je isključeno 19 vozača zbog uprava i kako nije projektila na procesu prethodnog procesa koji je na parkinu i predsjednik moralo da se protiv ministarstva unutrašnjih poslova koji su odbrani sa ukrajinima grada kao drugim glavnom p


In [ ]:
extend_text(telst, temperature=0.2)

Osim toga, u biltenu MUP-a KS se navodi da su u toku proteklog dana, službenici policije, preduzimajući redovne mjere kontrole saobraćaja, uručili su 554 prekršajna naloga, iz saobraćaja je isključeno 19 vozača zbog upravljanja protiv miliona u sarajevu i izbjeglice su predstavlja predstavnika postavlja se na startu kao i posljednjih poslova u posljednji se da se na takmičenju i protiv predsjednika bit će se od 100


In [ ]:
extend_text(telst, temperature=0.0001)

Osim toga, u biltenu MUP-a KS se navodi da su u toku proteklog dana, službenici policije, preduzimajući redovne mjere kontrole saobraćaja, uručili su 554 prekršajna naloga, iz saobraćaja je isključeno 19 vozača zbog upravljanja protiv predsjednika protiv selekcije bih u sarajevu i predsjednika bilo je potvrdio da su se do posljednjih poslova u sarajevu i predsjednika bilo je potvrdio da su se do posljednjih poslova


In [ ]:
extend_text(telst, temperature=0.01)

Osim toga, u biltenu MUP-a KS se navodi da su u toku proteklog dana, službenici policije, preduzimajući redovne mjere kontrole saobraćaja, uručili su 554 prekršajna naloga, iz saobraćaja je isključeno 19 vozača zbog upravljanja protiv predsjednika protiv selekcije bih u sarajevu i predsjednika bilo je potvrdio da su se do posljednjih poslova u sarajevu i predsjednika bilo je potvrdio da su se do posljednjih poslova


In [ ]:
tekat="""Ali, želja Ukrajine za članstvom u EU posebno je stvorila opipljiv strah na zapadnom Balkanu da će biti ostavljena po strani. Srbija ne želi da ima ništa sa NATO-om, a njen blizak odnos sa Moskvom zakomplikovao je nastojanja Beograda za ulazak u EU, još više od ruske invazije velikih razmjera na Ukrajinu.

Da nije bilo invazije na Ukrajinu, pristupni pregovori sa Albanijom i Sjevernom Makedonijom zasigurno bi zaglavili, a Bosna i Hercegovina ne bi bila priznata kao kandidat za EU. Možda se i EU ne bi složila oko budžeta za svoj novi plan rasta od šest milijardi eura za Zapadni Balkan. Plan uslovljava evropska ulaganja reformama na Balkanu, ali ako se ostvari njegov puni potencijal, zemlje u regionu mogle bi dobiti po glavi stanovnika skoro onoliko nov"""
print(extend_text(tekat, temperature=0.5))

Ali, želja Ukrajine za članstvom u EU posebno je stvorila opipljiv strah na zapadnom Balkanu da će biti ostavljena po strani. Srbija ne želi da ima ništa sa NATO-om, a njen blizak odnos sa Moskvom zakomplikovao je nastojanja Beograda za ulazak u EU, još više od ruske invazije velikih razmjera na Ukrajinu.

Da nije bilo invazije na Ukrajinu, pristupni pregovori sa Albanijom i Sjevernom Makedonijom zasigurno bi zaglavili, a Bosna i Hercegovina ne bi bila priznata kao kandidat za EU. Možda se i EU ne bi složila oko budžeta za svoj novi plan rasta od šest milijardi eura za Zapadni Balkan. Plan uslovljava evropska ulaganja reformama na Balkanu, ali ako se ostvari njegov puni potencijal, zemlje u regionu mogle bi dobiti po glavi stanovnika skoro onoliko novinarske lige je ministarstva na svoje druge šta je na sjevernoj putu predmeta u procesu svoje potpuno da je samo da se različita pod svemirskom kompanijom za od posljednjeg zakona i to su planira da s
None
